# Penjadwalan Kuliah dengan Algoritma Genetika
Notebook ini menyusun jadwal kuliah multi-kelas menggunakan pendekatan Genetic Algorithm (GA).


## Konteks dan aturan singkat
- Jadwal untuk semester 1, 3, 5, dan 7; masing-masing memiliki kelas A dan B.
- Hari kuliah: Senin?Sabtu, jam 07.00?17.00 dengan slot 50 menit (12 slot per hari).
- Ruang tersedia: TI1?TI6; satu ruang hanya untuk satu mata kuliah pada slot yang sama.
- Dosen luar biasa hanya boleh mengajar Jumat atau Sabtu.
- GA: populasi kromosom (jadwal), seleksi, crossover, mutasi; fitness menghukum benturan ruangan/dosen/kelas, pelanggaran dosen luar biasa, dan ketidakselarasan durasi SKS.


## Format file `mata_kuliah.csv`
Kolom wajib: `kode_mk, nama_mk, semester, kelas, dosen, tipe_dosen, sks`.
Contoh baris:

```
kode_mk,nama_mk,semester,kelas,dosen,tipe_dosen,sks
TI101,Kalkulus I,1,A,Dr. A,Tetap,3
TI103,Pengantar TI,1,B,Ir. B,LuarBiasa,2
TI203,Struktur Data,3,A,Dr. C,Tetap,3
```
Jika file tidak ditemukan, notebook menampilkan pesan dan memakai data sampel agar sel tetap dapat dieksekusi.


In [4]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import defaultdict
from typing import List, Dict, Tuple

plt.style.use("ggplot")
random.seed(42)
np.random.seed(42)


In [5]:
DAYS = ["Senin", "Selasa", "Rabu", "Kamis", "Jumat", "Sabtu"]
ROOMS = [f"TI{i}" for i in range(1, 7)]
SLOT_DURATION = 50  # menit

def build_time_slots(start_hour: int = 7, start_minute: int = 0, num_slots: int = 12, duration: int = 50):
    slots = []
    hour = start_hour
    minute = start_minute
    for idx in range(num_slots):
        end_minute = minute + duration
        end_hour = hour + end_minute // 60
        end_minute = end_minute % 60
        slots.append({
            "index": idx,
            "jam_mulai": f"{hour:02d}:{minute:02d}",
            "jam_selesai": f"{end_hour:02d}:{end_minute:02d}",
        })
        hour, minute = end_hour, end_minute
    return slots

TIME_SLOTS = build_time_slots()
TIME_SLOTS[:3]


[{'index': 0, 'jam_mulai': '07:00', 'jam_selesai': '07:50'},
 {'index': 1, 'jam_mulai': '07:50', 'jam_selesai': '08:40'},
 {'index': 2, 'jam_mulai': '08:40', 'jam_selesai': '09:30'}]

In [ ]:
REQUIRED_COLUMNS = [
    "kode_mk",
    "nama_mk",
    "semester",
    "kelas",
    "dosen",
    "tipe_dosen",
    "sks",
]


def read_course_data(path: str = "Daftar_matkul.csv") -> pd.DataFrame:
    try:
        df = pd.read_csv(path)
    except FileNotFoundError:
        print("File mata_kuliah.csv tidak ditemukan, pastikan file berada di direktori yang sama dengan notebook.")
        return None
    missing = [c for c in REQUIRED_COLUMNS if c not in df.columns]
    if missing:
        raise ValueError(f"Kolom wajib hilang: {missing}")
    return df


def make_sample_courses() -> pd.DataFrame:
    data = [
        {"kode_mk": "TI101", "nama_mk": "Kalkulus I", "semester": 1, "kelas": "A", "dosen": "Dr. A", "tipe_dosen": "Tetap", "sks": 3},
        {"kode_mk": "TI102", "nama_mk": "Aljabar Linear", "semester": 1, "kelas": "B", "dosen": "Dr. B", "tipe_dosen": "Tetap", "sks": 3},
        {"kode_mk": "TI103", "nama_mk": "Pengantar TI", "semester": 1, "kelas": "A", "dosen": "Ir. C", "tipe_dosen": "LuarBiasa", "sks": 2},
        {"kode_mk": "TI104", "nama_mk": "Logika Informatika", "semester": 1, "kelas": "B", "dosen": "Ir. D", "tipe_dosen": "Tetap", "sks": 2},
        {"kode_mk": "TI201", "nama_mk": "Struktur Data", "semester": 3, "kelas": "A", "dosen": "Dr. E", "tipe_dosen": "Tetap", "sks": 3},
        {"kode_mk": "TI202", "nama_mk": "Sistem Operasi", "semester": 3, "kelas": "B", "dosen": "Dr. F", "tipe_dosen": "Tetap", "sks": 3},
        {"kode_mk": "TI203", "nama_mk": "Pemrograman Berorientasi Objek", "semester": 3, "kelas": "A", "dosen": "Ir. G", "tipe_dosen": "LuarBiasa", "sks": 3},
        {"kode_mk": "TI204", "nama_mk": "Basis Data", "semester": 3, "kelas": "B", "dosen": "Dr. H", "tipe_dosen": "Tetap", "sks": 3},
        {"kode_mk": "TI301", "nama_mk": "Rekayasa Perangkat Lunak", "semester": 5, "kelas": "A", "dosen": "Dr. I", "tipe_dosen": "Tetap", "sks": 3},
        {"kode_mk": "TI302", "nama_mk": "Jaringan Komputer", "semester": 5, "kelas": "B", "dosen": "Ir. J", "tipe_dosen": "Tetap", "sks": 3},
        {"kode_mk": "TI303", "nama_mk": "Kecerdasan Buatan", "semester": 5, "kelas": "A", "dosen": "Ir. K", "tipe_dosen": "LuarBiasa", "sks": 3},
        {"kode_mk": "TI304", "nama_mk": "Interaksi Manusia dan Komputer", "semester": 5, "kelas": "B", "dosen": "Dr. L", "tipe_dosen": "Tetap", "sks": 2},
        {"kode_mk": "TI401", "nama_mk": "Manajemen Proyek TI", "semester": 7, "kelas": "A", "dosen": "Dr. M", "tipe_dosen": "Tetap", "sks": 3},
        {"kode_mk": "TI402", "nama_mk": "Keamanan Informasi", "semester": 7, "kelas": "B", "dosen": "Ir. N", "tipe_dosen": "Tetap", "sks": 3},
        {"kode_mk": "TI403", "nama_mk": "Data Mining", "semester": 7, "kelas": "A", "dosen": "Ir. O", "tipe_dosen": "LuarBiasa", "sks": 2},
        {"kode_mk": "TI404", "nama_mk": "Analitik Data", "semester": 7, "kelas": "B", "dosen": "Dr. P", "tipe_dosen": "Tetap", "sks": 2},
    ]
    return pd.DataFrame(data)


data_mk = read_course_data("Daftar_matkul.csv")
if data_mk is None:
    data_mk = make_sample_courses()
    print(f"Data sampel digunakan sebanyak {len(data_mk)} mata kuliah.")
else:
    print(f"Berhasil memuat {len(data_mk)} mata kuliah dari file.")

data_mk.head()


ParserError: Error tokenizing data. C error: Expected 2 fields in line 3, saw 3


In [ ]:
def random_gene(course_idx: int, course_df: pd.DataFrame) -> Dict:
    course = course_df.iloc[course_idx]
    sks = int(course["sks"])
    max_start = len(TIME_SLOTS) - sks
    start_slot = random.randint(0, max_start)
    day = random.randint(0, len(DAYS) - 1)
    room = random.choice(ROOMS)
    return {
        "course_idx": course_idx,
        "day": day,
        "start_slot": start_slot,
        "room": room,
    }


def random_chromosome(course_df: pd.DataFrame) -> List[Dict]:
    return [random_gene(i, course_df) for i in range(len(course_df))]


def init_population(pop_size: int, course_df: pd.DataFrame) -> List[List[Dict]]:
    return [random_chromosome(course_df) for _ in range(pop_size)]


def decode_chromosome(chromosome: List[Dict], course_df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for gene in chromosome:
        course = course_df.iloc[gene["course_idx"]]
        sks = int(course["sks"])
        start_slot = gene["start_slot"]
        end_slot = start_slot + sks - 1
        start_time = TIME_SLOTS[start_slot]["jam_mulai"] if start_slot < len(TIME_SLOTS) else "?"
        end_time = TIME_SLOTS[end_slot]["jam_selesai"] if end_slot < len(TIME_SLOTS) else "?"
        rows.append({
            "hari": DAYS[gene["day"]],
            "hari_index": gene["day"],
            "slot_mulai": start_slot,
            "slot_selesai": end_slot,
            "jam_mulai": start_time,
            "jam_selesai": end_time,
            "ruang": gene["room"],
            "kode_mk": course["kode_mk"],
            "nama_mk": course["nama_mk"],
            "semester": course["semester"],
            "kelas": course["kelas"],
            "dosen": course["dosen"],
            "tipe_dosen": course["tipe_dosen"],
            "sks": sks,
        })
    return pd.DataFrame(rows)


## Skema fitness
Fitness diukur sebagai `fitness = base_score - total_penalti` (tidak kurang dari 0).
Penalti yang digunakan:
- Benturan ruangan, dosen, dan kelas (semester+kelas) pada hari+slot yang sama.
- Dosen luar biasa mengajar di Senin?Kamis (penalti besar).
- Durasi tidak pas dengan SKS, slot melewati batas harian, atau tidak kontigu.
- Penalti ringan bila beban satu kelas per hari melewati 8 SKS (opsional soft-constraint).
Angka bobot dapat diubah sesuai kebutuhan.


In [ ]:
def compute_penalty(chromosome: List[Dict], course_df: pd.DataFrame) -> float:
    penalty = 0
    room_usage = defaultdict(list)
    lecturer_usage = defaultdict(list)
    class_usage = defaultdict(list)
    load_usage = defaultdict(int)

    for gene in chromosome:
        course = course_df.iloc[gene["course_idx"]]
        sks = int(course["sks"])
        day = gene["day"]
        start_slot = gene["start_slot"]
        end_slot = start_slot + sks - 1
        room = gene["room"]
        dosen = course["dosen"]
        kelas = (course["semester"], course["kelas"])

        if start_slot < 0 or end_slot >= len(TIME_SLOTS):
            penalty += 3000
            continue

        slots = list(range(start_slot, start_slot + sks))
        for idx in range(len(slots) - 1):
            if slots[idx + 1] - slots[idx] != 1:
                penalty += 3000
                break

        for slot in slots:
            room_usage[(day, slot, room)].append(course["kode_mk"])
            lecturer_usage[(day, slot, dosen)].append(course["kode_mk"])
            class_usage[(day, slot, kelas)].append(course["kode_mk"])

        if str(course["tipe_dosen"]).lower() == "luarbiasa" and day < 4:
            penalty += 5000

        load_usage[(day, kelas)] += sks

    for usage in room_usage.values():
        if len(usage) > 1:
            penalty += (len(usage) - 1) * 1200
    for usage in lecturer_usage.values():
        if len(usage) > 1:
            penalty += (len(usage) - 1) * 1200
    for usage in class_usage.values():
        if len(usage) > 1:
            penalty += (len(usage) - 1) * 1200

    for (day, kelas), beban in load_usage.items():
        if beban > 8:
            penalty += (beban - 8) * 10

    return penalty


def fitness(chromosome: List[Dict], course_df: pd.DataFrame, base_score: float = 10000.0) -> float:
    return max(0.0, base_score - compute_penalty(chromosome, course_df))


In [ ]:
def roulette_selection(population: List[List[Dict]], course_df: pd.DataFrame) -> List[Dict]:
    fitnesses = [fitness(chrom, course_df) for chrom in population]
    total_fit = sum(fitnesses)
    if total_fit == 0:
        return random.choice(population)
    pick = random.uniform(0, total_fit)
    current = 0
    for chrom, fit in zip(population, fitnesses):
        current += fit
        if current >= pick:
            return chrom
    return population[-1]


def tournament_selection(population: List[List[Dict]], course_df: pd.DataFrame, tournament_size: int = 3) -> List[Dict]:
    contenders = random.sample(population, k=min(tournament_size, len(population)))
    contenders.sort(key=lambda c: fitness(c, course_df), reverse=True)
    return contenders[0]


def one_point_crossover(parent1: List[Dict], parent2: List[Dict]) -> Tuple[List[Dict], List[Dict]]:
    if len(parent1) <= 1:
        return parent1.copy(), parent2.copy()
    point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2


def mutate(chromosome: List[Dict], course_df: pd.DataFrame, mutation_rate: float = 0.1) -> List[Dict]:
    new_chrom = []
    for gene in chromosome:
        if random.random() < mutation_rate:
            course = course_df.iloc[gene["course_idx"]]
            sks = int(course["sks"])
            max_start = len(TIME_SLOTS) - sks
            mutate_choice = random.choice(["day", "slot", "room"])
            if mutate_choice == "day":
                gene = {**gene, "day": random.randint(0, len(DAYS) - 1)}
            elif mutate_choice == "slot":
                gene = {**gene, "start_slot": random.randint(0, max_start)}
            else:
                gene = {**gene, "room": random.choice(ROOMS)}
        new_chrom.append(gene)
    return new_chrom


In [ ]:
def simple_ga(course_df: pd.DataFrame, population_size: int = 80, num_generations: int = 100, crossover_rate: float = 0.8, mutation_rate: float = 0.1):
    population = init_population(population_size, course_df)
    history = []
    best = None
    best_fitness = -1

    for gen in range(num_generations):
        fitness_values = [fitness(chrom, course_df) for chrom in population]
        history.append(max(fitness_values))
        best_idx = int(np.argmax(fitness_values))
        if fitness_values[best_idx] > best_fitness:
            best_fitness = fitness_values[best_idx]
            best = population[best_idx]

        new_population = []
        while len(new_population) < population_size:
            parent1 = roulette_selection(population, course_df)
            parent2 = roulette_selection(population, course_df)
            if random.random() < crossover_rate:
                child1, child2 = one_point_crossover(parent1, parent2)
            else:
                child1, child2 = parent1.copy(), parent2.copy()
            child1 = mutate(child1, course_df, mutation_rate)
            child2 = mutate(child2, course_df, mutation_rate)
            new_population.extend([child1, child2])
        population = new_population[:population_size]
    return best, best_fitness, history


def elitism_ga(course_df: pd.DataFrame, population_size: int = 80, num_generations: int = 100, crossover_rate: float = 0.8, mutation_rate: float = 0.1, elitism_rate: float = 0.05):
    population = init_population(population_size, course_df)
    history = []
    elite_count = max(1, int(elitism_rate * population_size))
    best = None
    best_fitness = -1

    for gen in range(num_generations):
        fitness_values = [fitness(chrom, course_df) for chrom in population]
        sorted_pop = [chrom for _, chrom in sorted(zip(fitness_values, population), key=lambda x: x[0], reverse=True)]
        elites = sorted_pop[:elite_count]
        history.append(fitness_values[np.argmax(fitness_values)])

        if history[-1] > best_fitness:
            best_fitness = history[-1]
            best = sorted_pop[0]

        new_population = elites.copy()
        while len(new_population) < population_size:
            parent1 = roulette_selection(population, course_df)
            parent2 = roulette_selection(population, course_df)
            if random.random() < crossover_rate:
                child1, child2 = one_point_crossover(parent1, parent2)
            else:
                child1, child2 = parent1.copy(), parent2.copy()
            child1 = mutate(child1, course_df, mutation_rate)
            child2 = mutate(child2, course_df, mutation_rate)
            new_population.extend([child1, child2])
        population = new_population[:population_size]
    return best, best_fitness, history


def tournament_ga(course_df: pd.DataFrame, population_size: int = 80, num_generations: int = 100, crossover_rate: float = 0.8, mutation_rate: float = 0.1, tournament_size: int = 3):
    population = init_population(population_size, course_df)
    history = []
    best = None
    best_fitness = -1

    for gen in range(num_generations):
        fitness_values = [fitness(chrom, course_df) for chrom in population]
        gen_best = max(fitness_values)
        history.append(gen_best)
        if gen_best > best_fitness:
            best_fitness = gen_best
            best = population[int(np.argmax(fitness_values))]

        new_population = []
        while len(new_population) < population_size:
            parent1 = tournament_selection(population, course_df, tournament_size)
            parent2 = tournament_selection(population, course_df, tournament_size)
            if random.random() < crossover_rate:
                child1, child2 = one_point_crossover(parent1, parent2)
            else:
                child1, child2 = parent1.copy(), parent2.copy()
            child1 = mutate(child1, course_df, mutation_rate)
            child2 = mutate(child2, course_df, mutation_rate)
            new_population.extend([child1, child2])
        population = new_population[:population_size]
    return best, best_fitness, history


In [ ]:
def run_ga(
    course_df: pd.DataFrame,
    method: str = "simple_ga",
    population_size: int = 100,
    num_generations: int = 150,
    crossover_rate: float = 0.85,
    mutation_rate: float = 0.1,
    elitism_rate: float = 0.05,
    tournament_size: int = 3,
):
    if method == "simple_ga":
        return simple_ga(course_df, population_size, num_generations, crossover_rate, mutation_rate)
    if method == "elitism_ga":
        return elitism_ga(course_df, population_size, num_generations, crossover_rate, mutation_rate, elitism_rate)
    if method == "tournament_ga":
        return tournament_ga(course_df, population_size, num_generations, crossover_rate, mutation_rate, tournament_size)
    raise ValueError("Metode GA tidak dikenali")


best_chromosome, best_score, history = run_ga(
    data_mk,
    method="elitism_ga",
    population_size=120,
    num_generations=120,
    crossover_rate=0.9,
    mutation_rate=0.15,
    elitism_rate=0.08,
    tournament_size=3,
)

print(f"Fitness terbaik: {best_score:.2f}")


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(history, label="Fitness terbaik per generasi")
plt.xlabel("Generasi")
plt.ylabel("Fitness")
plt.title("Kinerja Genetic Algorithm")
plt.legend()
plt.tight_layout()
plt.show()

jadwal_terbaik_df = decode_chromosome(best_chromosome, data_mk)
jadwal_terbaik_df = jadwal_terbaik_df.sort_values(["hari_index", "slot_mulai", "ruang"]).reset_index(drop=True)
jadwal_terbaik_df


In [ ]:
output_file = "jadwal_terbaik.csv"
jadwal_terbaik_df.to_csv(output_file, index=False)
print(f"Jadwal terbaik disimpan ke {output_file}")


## Kapan memilih tiap metode GA?
- `simple_ga`: baseline simpel, cocok untuk eksperimen awal atau populasi besar dengan komputasi ringan.
- `elitism_ga`: gunakan ketika ingin menjaga individu unggul agar tidak hilang; baik saat fitness fluktuatif.
- `tournament_ga`: cocok bila ingin tekanan seleksi lebih kuat/terkendali lewat `tournament_size`; efektif di landscape fitness yang datar.
